### 第4章画像分類（その4）
#### 精度向上のテクニック

In [1]:
from collections import deque
import copy
from tqdm import tqdm
from PIL import Image
from pathlib import Path

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
import torchvision.transforms as T

from src import utils, model

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# パラメータ初期化の追加
"""
パラメータ初期化関数
"""
def _reset_parameters(self):
    for m in self.modules():
        if isinstance(m, nn.Conv2d):
            # Heらが提案した正規分布を使って初期化
            nn.init.kaiming_normal_(m.weight, mode="fan_in",
                                    nonlinearity="relu")

In [3]:
# ResNet18の実装
class ResNet18(nn.Module):
    """
    ResNet18モデル
    num_classes : 分類対象の物体モデル数
    """
    def __init__(self, num_classes: int):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2,
                               padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = nn.Sequential(
            model.BasicBlock(64, 64),
            model.BasicBlock(64, 64),
        )
        self.layer2 = nn.Sequential(
            model.BasicBlock(64, 128, stride=2),
            model.BasicBlock(128, 128),
        )
        self.layer3 = nn.Sequential(
            model.BasicBlock(128, 256, stride=2),
            model.BasicBlock(256, 256),
        )
        self.layer4 = nn.Sequential(
            model.BasicBlock(256, 512, stride=2),
            model.BasicBlock(512, 512),
        )
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        # ドロップアウトの追加
        self.dropout = nn.Dropout()
        
        self.linear = nn.Linear(512, num_classes)
        
        self._reset_parameters()
    
    """
    パラメータの初期化関数
    """
    def _reset_parameters(self):
        for m in self.modules:
            if isinstance(m, nn.Conv2d):
                # Heらが考案した正規分布を使って初期化
                nn.init.kaiming_normal_(m.weight, mode='fan_in',
                                        nonlinearity='relu')
    
    """
    順伝播関数
    x            : 入力, [バッチサイズ, 入力チャネル数, 高さ, 幅]
    return_embed : 特徴量を返すかロジットを返すかを選択する真偽値
    """
    def forward(self, x: torch.Tensor, return_embed: bool=False):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avg_pool(x)
        x = x.flatten(1)
        
        if return_embed:
            return x
        
        x = self.dropout(x)
        
        x = self.linear(x)
        
        return x

In [1]:
# 学習におけるハイパーパラメータの設定
class Config:
    '''
    ハイパーパラメータとオプションの設定
    '''
    def __init__(self):
        self.val_ratio = 0.2       # 検証に使う学習セット内のデータの割合
        self.patch_size = 4        # パッチサイズ
        self.dim_hidden = 512      # 隠れ層の次元
        self.num_heads = 8         # マルチヘッドアテンションのヘッド数
        self.dim_feedforward = 512 # Transformerエンコーダ層内のFNNにおける隠れ層の特徴量次元
        self.num_layers = 6        # Transformerエンコーダの層数
        self.num_epochs = 30       # 学習エポック数
        self.lr = 1e-2             # 学習率
        self.moving_avg = 20       # 移動平均で計算する損失と正確度の値の数
        self.batch_size = 32       # バッチサイズ
        self.num_workers = 2       # データローダに使うCPUプロセスの数
        self.device = 'cuda'       # 学習に使うデバイス
        self.num_samples = 200     # t-SNEでプロットするサンプル数